Test

In [18]:
# Cellule: imports et chargement de la clé API depuis .env
import os
from dotenv import load_dotenv   # charge les variables d'environnement depuis un fichier .env
import requests                 # pour les appels HTTP
import pandas as pd             # manipulation de données tabulaires
import json                     # pour afficher proprement le JSON si besoin

# Charge les variables d'environnement depuis le fichier .env (s'il existe).
load_dotenv()

# Récupère la clé API depuis la variable d'environnement FOOTBALL_DATA_TOKEN
API_TOKEN = os.getenv("FOOTBALL_DATA_TOKEN")

# Vérification simple : on lève une erreur explicite si la clé n'est pas trouvée
if not API_TOKEN:
    raise RuntimeError("Clé API non trouvée. Crée un fichier .env contenant FOOTBALL_DATA_TOKEN=ta_cle")

# Base URL pour football-data.org (version v4)
BASE_URL = "https://api.football-data.org/v4"

# Prépare les headers d'authentification pour toutes les requêtes
HEADERS = {"X-Auth-Token": API_TOKEN}

# Optionnel : petite info pour l'utilisateur
print("Clé API chargée. Base:", BASE_URL)


Clé API chargée. Base: https://api.football-data.org/v4


In [19]:
# Cellule: wrapper simple pour faire des requêtes GET vers l'API
def api_get(path, params=None, base=BASE_URL, headers=HEADERS, timeout=10):
    """
    Effectue une requête GET sur base + path, retourne le JSON décodé.
    - path : chemin après la base (ex: "/competitions/2021/standings")
    - params : dict de query params (facultatif)
    - headers : en-têtes HTTP (par défaut HEADERS préparés ci-dessus)
    """
    url = base.rstrip("/") + "/" + path.lstrip("/")  # construit correctement l'URL
    # 1) On envoie la requête
    r = requests.get(url, headers=headers, params=params, timeout=timeout)
    # 2) Si le statut n'est pas 200, raise_for_status déclenche une exception HTTPError
    r.raise_for_status()
    # 3) Retourne la réponse décodée en JSON (sous forme de dict / list selon la réponse)
    return r.json()


In [24]:
# Cellule: obtenir le classement (standings) d'une compétition
competition_id = 2015  # exemple: 2015 = Ligue 1 (selon football-data.org), 2021 = Premier League

# Appel de l'API pour récupérer les standings
matches_json = api_get(f"/competitions/{competition_id}/matches",params={"season": 2025})

# Inspecte la structure (affiche les clés principales)
print("Clés du JSON:", matches_json.keys())

# Pour voir un extrait proprement :
print(json.dumps(matches_json, indent=2)[:500])  # affiche les 1000 premiers caractères


Clés du JSON: dict_keys(['filters', 'resultSet', 'competition', 'matches'])
{
  "filters": {
    "season": 2025
  },
  "resultSet": {
    "count": 306,
    "first": "2025-08-15",
    "last": "2026-05-16",
    "played": 108
  },
  "competition": {
    "id": 2015,
    "name": "Ligue 1",
    "code": "FL1",
    "type": "LEAGUE",
    "emblem": "https://crests.football-data.org/FL1.png"
  },
  "matches": [
    {
      "area": {
        "id": 2081,
        "name": "France",
        "code": "FRA",
        "flag": "https://crests.football-data.org/773.svg"
      },
      "compet


In [ ]:
def pts(row):
    if row["score.winner"] == "HOME_TEAM" and row["is_home"]:
        return 3
    if row["score.winner"] == "AWAY_TEAM" and not row["is_home"]:
        return 3
    if row["score.winner"] == "DRAW":
        return 1
    return 0

def classement_interactif(matches_json):
    matches_list = matches_json["matches"]
    df_short = pd.json_normalize(matches_list)[["id","status","matchday","season.id","homeTeam.name","awayTeam.name","score.winner","score.fullTime.home","score.fullTime.away"]
]

    df_home = df_short.copy()
    df_home["team"] = df_home["homeTeam.name"]
    df_home["gf"] = df_home["score.fullTime.home"]
    df_home["ga"] = df_home["score.fullTime.away"]
    df_home["is_home"] = True

    df_away = df_short.copy()
    df_away["team"] = df_away["awayTeam.name"]
    df_away["gf"] = df_away["score.fullTime.away"]
    df_away["ga"] = df_away["score.fullTime.home"]
    df_away["is_home"] = False
    df_long = pd.concat([df_home, df_away], ignore_index=True)



    df_long["points"] = df_long.apply(pts, axis=1)
    df_long["win"]   = df_long["points"] == 3
    df_long["draw"]  = df_long["points"] == 1
    df_long["loss"]  = df_long["points"] == 0



array(['Olympique de Marseille', 'Olympique Lyonnais', 'Le Havre AC',
       'Toulouse FC', 'Lille OSC', 'FC Lorient', 'RC Strasbourg Alsace',
       'Paris FC', 'Paris Saint-Germain FC', 'Angers SCO', 'AJ Auxerre',
       'FC Metz', 'Stade Rennais FC 1901', 'Racing Club de Lens',
       'Stade Brestois 29', 'FC Nantes', 'AS Monaco FC', 'OGC Nice'],
      dtype=object)